In [1]:
from util.backtest import *
import matplotlib.pyplot as plt
from matplotlib import cm
import pymongo
import numba

plt.style.use('seaborn')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
plt.rcParams['axes.unicode_minus'] =False

In [2]:
INSAMPLE_END_DATE = datetime(2021,7,1)
VALID_END_DATE = datetime(2022,6,1)
SAMPLE_END_DATE = datetime(2022,8,31)

FREQ = 'D'


df = pd.read_csv('/home/frank/document/Python/Factors/data/data_1h.csv')
df['BuyerRatio'] = df['takerBuyQuoteVol']/df['quoteAssetVolume']
df['BuyerPerTrade'] = df['takerBuyQuoteVol']/df['numberOfTrades']
df['VolumePerTrade'] = df['quoteAssetVolume']/df['numberOfTrades']

df = df.drop(['Volume','numberOfTrades'],axis= 1)
print('Dataframe prepared completed')

Dataframe prepared completed


In [3]:
col_list = ['Open','High','Low','Close','quoteAssetVolume','takerBuyQuoteVol','BuyerRatio']
df_data = {}
for col in col_list:
    df_data[col] = df.pivot(values = col,index = 'openTime',columns = 'symbol').astype(float)
    df_data[col].index = pd.to_datetime(df_data[col].index,unit = 'ms') #+ timedelta(hours=8)

print("Pivot table completed")

Pivot table completed


In [4]:
close = df_data['Close']#.resample('D').last()
close.tail()

symbol,1000bttcusdt,1000luncusdt,1000shibusdt,1000xecusdt,1inchusdt,aaveusdt,achusdt,adausdt,agixusdt,akrousdt,...,xlmusdt,xmrusdt,xrpusdt,xtzusdt,yfiiusdt,yfiusdt,zecusdt,zenusdt,zilusdt,zrxusdt
openTime,,,,,,,,,,,,,,,,,,,,,
2023-02-23 03:00:00,NaN,0.1649,0.013403,0.03745,0.6357,86.42,0.04472,0.3917,0.4180,NaN,...,0.09265,157.90,0.3951,1.443,NaN,9077.0,47.96,13.816,0.03573,0.2721
2023-02-23 04:00:00,NaN,0.1655,0.013481,0.03760,0.6395,87.05,0.04416,0.3924,0.4198,NaN,...,0.09268,157.88,0.3969,1.454,NaN,9505.0,48.00,13.730,0.03603,0.2762
2023-02-23 05:00:00,NaN,0.1641,0.013490,0.03726,0.6383,86.60,0.04779,0.3909,0.4166,NaN,...,0.09218,157.23,0.3957,1.437,NaN,9506.0,47.80,13.802,0.03563,0.2730
2023-02-23 06:00:00,NaN,0.1640,0.013424,0.03706,0.6370,86.61,0.05046,0.3912,0.4182,NaN,...,0.09221,157.18,0.3941,1.417,NaN,9463.0,47.81,13.741,0.03571,0.2723
2023-02-23 07:00:00,NaN,0.1646,0.013400,0.03720,0.6386,86.83,0.05036,0.3930,0.4262,NaN,...,0.09245,157.60,0.3949,1.416,NaN,9367.0,47.93,13.812,0.03570,0.2723


In [5]:
df1 = close.values
len(df1[-1:][0])
df1.shape

(27592, 182)

In [6]:
%%time


@numba.njit(parallel=True,nogil=True)
def nb_rolling_max(arr,periods):
    result = np.empty(arr.shape,dtype=np.float64)

    for col in nb.prange(arr.shape[0]):
        for i in range(periods,arr.shape[1]+1):
            target = arr[col][i-periods:i].copy()
            moments = np.std(target)
            if not np.all(np.isnan(target)) and moments != 0:
                m2 = moments**2
                m3 = moments**3
                n = periods

                result[col][i-1] = (np.sqrt(n*(n-1))/ (n-2)) * (m3 / m2 ** 1.5)

    return result
# close
# ret[-1]
ret = nb_rolling_max(df1[:,:].T,5)#.T
ret[-1,-5:]
# pd.DataFrame(ret)[0].plot()

CPU times: user 1min 28s, sys: 111 ms, total: 1min 28s
Wall time: 2.2 s


array([1.49071198, 1.49071198, 1.49071198, 1.49071198, 1.49071198])

In [12]:
# moments(close.values,2)
skewness(close['btcusdt'].values)

1.0000543673602615

In [287]:


@nb.njit(parallel=True)
def nb_moments(values, moment):
    arr = values[~np.isnan(values)]
    if arr.size == 0:
        return np.NaN

    vals_mean = np.mean(arr)
    numerator = (arr - vals_mean) ** moment
    numerator = np.mean(numerator )
    return numerator

def skewness(values): 
    n = len(values)
    m2 = np.std(values,ddof= 0)**2
    m3 = np.std(values,ddof= 0)**3
    return (np.sqrt(n*(n-1))/ (n-2)) * (m3 / m2 ** 1.5)
    
@nb.njit(parallel=True)
def nb_kurtosis(values):
    n = len(values)
    m2 = nb_moments(values,2)
    m4 = nb_moments(values,4)
    nval = 1.0/(n-2)/(n-3) * ((n**2-1.0)*m4/m2**2.0 - 3*(n-1)**2.0)
    return nval
# _moment(a, 4, axis, mean=mean)

In [284]:
@nb.njit(parallel=True)
def nb_test(values):
    mask = np.isnan(values)
    arr = values[~mask]
    return np.mean(values[~np.isnan(values)])
nb_test(close['1000bttcusdt'].values)    

In [286]:
nb_test(close['1000bttcusdt'].values)
close['1000bttcusdt'].values.size

27592

In [350]:
def nb_rolling(arr,periods,method,params = None):
    result = np.empty( arr.shape,dtype=np.float64)
    if len(arr.shape)>1:
        for col in nb.prange(arr.shape[0]):
            for i in range(periods,arr.shape[1]+1):
                target = arr[col][i-periods:i].copy()
                result[col][i-1] = nb_kurtosis(target)
    else:
        for i in range(periods,arr.shape[0]+1):
            target = arr[i-periods:i].copy()
            if params:
                result[i-1] = method(target[target!=np.nan],*params)
            else:
                result[i-1] = method(target[target!=np.nan])

    return result
@nb.njit(parallel=True)

def test_func(x,y):
    return 1
result = nb_rolling(close['1000bttcusdt'].values,close['1000bttcusdt'].values.size - 10,test_func,(20,))
result

/home/frank/.local/lib/python3.10/site-packages/numba/core/typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../tmp/ipykernel_3155991/3751634493.py", line 17:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


array([6.92663421e-310, 6.92663421e-310, 4.66610616e-310, ...,
       1.00000000e+000, 1.00000000e+000, 1.00000000e+000])

In [ ]:
result = nb_rolling(close['1000bttcusdt'].values,close['1000bttcusdt'].values.size - 10,test_func,(20,))


In [313]:
result#[:10]

array([        nan,         nan,         nan, ..., -1.09788305,
       -1.09788305, -1.09788305])

In [292]:
%%time
# nb_moments(close['ethusdt'].values,2)
nb_kurtosis(close['1000bttcusdt'].values)

CPU times: user 6.35 s, sys: 0 ns, total: 6.35 s
Wall time: 77.7 ms


-1.0978830556822639

In [293]:
close['1000bttcusdt'].kurt()

-1.0976177536599054

In [252]:
%%time
# _moment(close['ethusdt'].values,2,0)
close['1000bttcusdt'].kurt()

CPU times: user 2.05 ms, sys: 0 ns, total: 2.05 ms
Wall time: 1.52 ms


-1.0976177536599054

In [246]:
close#['ethusdt'].std()

symbol,1000bttcusdt,1000luncusdt,1000shibusdt,1000xecusdt,1inchusdt,aaveusdt,achusdt,adausdt,agixusdt,akrousdt,...,xlmusdt,xmrusdt,xrpusdt,xtzusdt,yfiiusdt,yfiusdt,zecusdt,zenusdt,zilusdt,zrxusdt
openTime,,,,,,,,,,,,,,,,,,,,,
2019-12-31 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-23 03:00:00,NaN,0.1649,0.013403,0.03745,0.6357,86.42,0.04472,0.3917,0.4180,NaN,...,0.09265,157.90,0.3951,1.443,NaN,9077.0,47.96,13.816,0.03573,0.2721
2023-02-23 04:00:00,NaN,0.1655,0.013481,0.03760,0.6395,87.05,0.04416,0.3924,0.4198,NaN,...,0.09268,157.88,0.3969,1.454,NaN,9505.0,48.00,13.730,0.03603,0.2762
2023-02-23 05:00:00,NaN,0.1641,0.013490,0.03726,0.6383,86.60,0.04779,0.3909,0.4166,NaN,...,0.09218,157.23,0.3957,1.437,NaN,9506.0,47.80,13.802,0.03563,0.2730


In [43]:
close['ethusdt'].skew()

0.5125483707449605

In [76]:
close['ethusdt'].kurt()


-0.743234790635865

In [82]:
from scipy.stats import kurtosis,skew,moment
skew(close['ethusdt'],bias=False)
test  ={}
for col in close.columns:
    test[col] = kurtosis(close[col].dropna())
test1 = pd.Series(test).rank()

In [71]:
test = {}
for col in close.columns:
    test[col] = close[col].dropna().kurt()
test2 = pd.Series(test).rank()

In [83]:
def _moment(a, moment, axis, *, mean=None):
    if np.abs(moment - np.round(moment)) > 0:
        raise ValueError("All moment parameters must be integers")

    # moment of empty array is the same regardless of order
    if a.size == 0:
        return np.mean(a, axis=axis)

    if moment == 0 or moment == 1:
        # By definition the zeroth moment about the mean is 1, and the first
        # moment is 0.
        shape = list(a.shape)
        del shape[axis]
        dtype = a.dtype.type if a.dtype.kind in 'fc' else np.float64

        if len(shape) == 0:
            return dtype(1.0 if moment == 0 else 0.0)
        else:
            return (np.ones(shape, dtype=dtype) if moment == 0
                    else np.zeros(shape, dtype=dtype))
    else:
        # Exponentiation by squares: form exponent sequence
        n_list = [moment]
        current_n = moment
        while current_n > 2:
            if current_n % 2:
                current_n = (current_n - 1) / 2
            else:
                current_n /= 2
            n_list.append(current_n)

        # Starting point for exponentiation by squares
        mean = a.mean(axis, keepdims=True) if mean is None else mean
        a_zero_mean = a - mean

        eps = np.finfo(a_zero_mean.dtype).resolution * 10
        with np.errstate(divide='ignore', invalid='ignore'):
            rel_diff = np.max(np.abs(a_zero_mean), axis=axis,
                              keepdims=True) / np.abs(mean)
        with np.errstate(invalid='ignore'):
            precision_loss = np.any(rel_diff < eps)
        if precision_loss:
            message = ("Precision loss occurred in moment calculation due to "
                       "catastrophic cancellation. This occurs when the data "
                       "are nearly identical. Results may be unreliable.")
            warnings.warn(message, RuntimeWarning, stacklevel=4)

        if n_list[-1] == 1:
            s = a_zero_mean.copy()
        else:
            s = a_zero_mean**2

        # Perform multiplications
        for n in n_list[-2::-1]:
            s = s**2
            if n % 2:
                s *= a_zero_mean
        return np.mean(s, axis)


In [74]:
test = (test1 - test2)
test[test!=0].describe()

count    48.000000
mean      0.000000
std       3.718842
min     -22.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
dtype: float64